In [1]:
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm # progress apply :)
tqdm.pandas()

%load_ext autoreload
%autoreload 2
def d(l):
    pd.set_option('display.max_colwidth', l)



In [3]:
new_articles=pd.read_excel("2025-04-26 Manual Annotation results_Inclusion.xlsx")["ID"].to_list()


In [15]:
# ensure that lists are read as list and not as strings
from ast import literal_eval
columns_to_convert=["texts","labels","retraction_counts","has_retraction","has_retraction_label","contains_country","country_plus_city",'fields']
converters = {col: literal_eval for col in columns_to_convert}

df_country_fields=pd.read_csv("2025-04-26 Country_fields results.csv",index_col=0, converters=converters)
df_dict=pd.read_csv("2025-04-26 data with dict_approach.csv",index_col=0, converters=converters)
df_GPT4=pd.read_csv('2025-04-03 full data with GPT4o-mini_response.csv', converters=converters)
df_GPT4["name"]=df_GPT4.path.str.split("\\").apply(lambda x: x[-1].split(".")[0])
df_dict['name']=df_dict.index

In [7]:
df_country_fields[df_country_fields.name.isin(new_articles)].shape[0]

16

In [64]:
for i in new_articles:
    print(i, i in df_country_fields.name)

Nature1223 False
Nature1358 True
Nature293 True
Science138 True
Science297 True
Science328 True
Science329 True
Science370 True
Science378 True
Science384 True
Science421 True
Science426 True
Science58 True
Science645 True
Science928 True
Science188 True
Science735 True


In [17]:
# Check if also the new articles are in the processed collections
print("GPT files",df_GPT4[df_GPT4.name.isin(new_articles)].shape[0],len(new_articles))
print("dict approach",df_dict[df_dict.name.isin(new_articles)].shape[0],len(new_articles))
print("country fields",df_country_fields[df_country_fields.name.isin(new_articles)].shape[0],len(new_articles))

GPT files 17 17
dict approach 17 17
country fields 16 17


In [19]:
df_meta_data=pd.read_excel("2025-04-25 all_Meta_Data.xlsx")[['ID', 'Doi_x', 'journal', 'Title', 'Date', 'Year', 'Type',
       'Type Recoded', 'confirmed_2024', '9. retained for analysis (revised)']]
df_meta_data.rename(columns={"ID": "name","Doi_x": "Doi"},inplace=True)

In [22]:
# Start with the first DataFrame

df_merged = df_meta_data

# Iteratively merge the remaining DataFrames on the "name" column
for df in [df_GPT4,df_country_fields ,df_dict]:
    df_merged = pd.merge(df_merged, df, on="name", how="outer")
# df_merged[df_merged["Year"].isna()][df_old.columns].to_excel("2025_02_04 missing_date_DOI_Type_recoded.xlsx")

In [52]:
df_merged.rename(columns={"text_x":"text"},inplace=True)
df=df_merged[df_merged["9. retained for analysis (revised)"]]

In [54]:
df.Year.min()

1963

# Add manual Annotation (classification)

In [56]:
df[["text","explanation"]].sample(10).to_excel("2025-04-26 Sample for manual annotation.xlsx")

In [58]:
import pandas as pd

# Load Excel files and set 'name' as the index
dfm = pd.read_excel("2025-02-24 Sample for manual annotation_filled.xlsx")[["name","Max code"]]
dfm2 = pd.read_excel("2025-03-07 Sample for manual annotation filled.xlsx")[["name","Max code"]]
dfm3 = pd.read_excel("2025-03-07 2nd Sample for manual annotation filled.xlsx")[["name","Max code"]]

# Concatenate DataFrames, drop rows with NaN in 'Max code', and remove duplicate indices
df_annotated = pd.concat([dfm, dfm2,dfm3], axis=0).dropna(subset=["Max code"])
df_annotated = df_annotated[~df_annotated.index.duplicated(keep="first")]



df_final=pd.merge(df,df_annotated,on="name",how="left")

In [60]:
#df_final=df_final.dropna(subset="Year")
df_final.sort_values("name").to_json("2025-04-26 Merged_Data.json", date_format='iso')
df_final.sort_values("name").to_excel("2025-04-26 Merged_Data.xlsx")

In [34]:
df_final["Date"]

0      2023-06-22
1      2021-06-09
2      2005-09-24
3      2005-09-21
4      2005-06-22
          ...    
1010   2022-03-24
1011   2022-03-24
1012   2022-02-03
1013   2021-11-25
1014   2021-11-04
Name: Date, Length: 1015, dtype: datetime64[ns]

In [64]:
df_final[df_final.text.isna()]

,name,Doi,journal_x,Title,Date,Year,Type,Type Recoded,confirmed_2024,9. retained for analysis (revised),...,NoStopwords,Nouns_Verbs,misconduct,mistake,retract,n_misconduct,n_mistake,n_retract,dict_topic,Max code
